In [10]:
import os
import shutil
%matplotlib inline

In [12]:
import pandas as pd

In [13]:
DIR = os.getcwd()
print DIR

/notebooks


In [14]:
#Data Preprocessing,move images to each sub repository for dog and cat
path = "/notebooks/tfv_data/dogscats"
train_path = path + "/train"
test_path = path + "/test"
valid_path = path + "/valid"

%cd train_path
%mkdir cat
%mkdir dog
%mv cat.*.jpg cat/
%mv dog.*.jpg dog/

%cd test_path
%mkdir cat
%mkdir dog
%mv cat.*.jpg cat/
%mv dog.*.jpg dog/

[Errno 2] No such file or directory: 'train_path'
/notebooks
mkdir: cannot create directory 'cat': File exists
mkdir: cannot create directory 'dog': File exists
mv: cannot stat 'cat.*.jpg': No such file or directory
mv: cannot stat 'dog.*.jpg': No such file or directory
[Errno 2] No such file or directory: 'test_path'
/notebooks
mkdir: cannot create directory 'cat': File exists
mkdir: cannot create directory 'dog': File exists
mv: cannot stat 'cat.*.jpg': No such file or directory
mv: cannot stat 'dog.*.jpg': No such file or directory


In [15]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py

In [16]:
#Export the Feature Vector
def setup_fc(MODEL, image_size, lambda_func=None):
    #[height,width,channel]
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    
    x = input_tensor
    if lambda_func :
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    #Print the model summary
    #model.summary()
    
    #Use the ImageDataGenerator to get enhance and regulized data 
    #Use mini bactch to train in each epoch
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_path, image_size, 
                                              shuffle=False,batch_size=16)
    test_generator = gen.flow_from_directory(test_path, image_size, 
                                             shuffle=False, batch_size=16, class_mode=None)
    
    train = model.predict_generator(train_generator, train_generator.nb_sample)
    test = model.predict_generator(test_generator, test_generator.nb_sample)
    
    with h5py.File("fc_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
     

In [17]:
#Import dogcat date set into pre-trained VGGNet16
setup_fc(VGG16, (224, 224))

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


RuntimeError: Unable to create link (Name already exists)

In [ ]:
#Import dogcat date set into pre-trained ResNet50
setup_fc(ResNet50, (224,224))

In [ ]:
#Import dogcat date set into pre-trained InceptionV3
setup_fc(InceptionV3, (299, 299), inception_v3.preprocess_input)

In [ ]:
#Import dogcat date set into pre-trained Xception
setup_fc(Xception, (299, 299), xception.preprocess_input)

In [ ]:
from keras import *
from keras.layers import *
from keras.models import *
from keras.regularizers import l2, activity_l2

In [ ]:
#载入特征向量
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)
X_train = []
X_test = []

for filename in ["fc_ResNet50.h5", "fc_InceptionV3.h5", "fc_Xception.h5", "fc_VGG16.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
#Build the ensemble model
#Import several learning rate optimzer to find the best one

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

Rmsp = keras.optimizers.RMSprop(lr=1e-3, rho=0.88, epsilon=1e-5)
model.compile(optimizer=Rmsp, loss='binary_crossentropy', metrics=['accuracy'])

#model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#nadam = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
#model.compile(optimizer=nadam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Export the visualization of the ensemble network 

from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
# train the model
model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

In [ ]:
#Predict the results 
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("/notebooks/tfv_data/sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_path, (224,224), shuffle=False,
                                         batch_size=32, class_mode=None)

for i, filename in enumerate(test_generator.filenames):
    index = int(filename[filename.rfind('/')+1:filename.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred0403_rmsp.csv', index=None)
df.head(10)